## What I want to achieve

Based on my current knowledge of python and machine learning, I want to be able to run an ensemble to algos to predict titanic survival. Gosh, we're still into that. I hope I dont ever have to touch this or the iris dataset in a years time...Thanks to [Bugra Akyildiz](http://bugra.github.io/work/notes/2014-11-22/an-introduction-to-supervised-learning-scikit-learn/) for a great blog on supervised learning. Check it out and weep with joy!

Ok, so what do we need to do? - Binary classification of survival

## Admin stuff

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None) # Display any number of columns

import numpy as np
from matplotlib import pyplot
%matplotlib inline

# Set seaborn aesthetic parameters to defaults
import seaborn as sns
sns.set()

/Users/Vivek/anaconda/envs/titanic_kaggle/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


## Read the training dataset

In [12]:
train_raw = pd.read_csv('train.csv')
train_raw.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


## Data wrangling from the lessons learned during the exploratory session

In [3]:
# quick test to see if passenger id is indeed unique and can be used as the index
print len(train.PassengerId), len(train.PassengerId.unique())

891 891


In [4]:
# fill up age variable with righteous values i.e. median age
print train.Age.isnull().sum()
train['Age'].fillna(train['Age'].median(), inplace = True)
print train.Age.isnull().sum()

177
0


In [31]:
# check how many rows have cabin level information
print train.Cabin.isnull().sum(), "pax dont have cabin level information"

# what about ticket info
print train.Ticket.isnull().sum(), "pax are travelling ticketless. However,...."

# however, multiple people could be travelling on the same ticket. Lets see if thats the case
print len(train.Ticket) - len(train.Ticket.drop_duplicates()), "pax are travelling on another passengers tickets."

687 pax dont have cabin level information
0 pax are travelling ticketless. However,....
210 pax are travelling on another passengers tickets.


In [33]:
def df_cleanup(raw_dataset):
    
    # convert the passengerid column into index. I've already checked that this field is unique per row.
    clean_dataset = raw_dataset.set_index('PassengerId', drop=True)
    
    # there's heaps of null ages. I got this from the exploratory session in the previous workbooks. Fill with 
    # median age of the passengers
    clean_dataset['Age'].fillna(clean_dataset['Age'].median(), inplace = True)
    
    # Drop redundant columns from the clean dataset
    # note that I'm removing the cabin information for now, because there is too much scope for overfitting here 
    clean_dataset.drop(['Name','Cabin'], axis=1, inplace=True)
    
    # Encode string values into discrete integers
    #import & instantiate
    from sklearn import preprocessing
    label_encoder = preprocessing.LabelEncoder()
    # fit & transform SEX variable into discrete integer values
    clean_dataset['Sex'] = label_encoder.fit_transform(clean_dataset['Sex'])
    # fit & transform TICKET variable into discrete integer values. Given 210 pax have dup tickets, this could be 
    # predictive.
    clean_dataset['Ticket'] = label_encoder.fit_transform(clean_dataset['Ticket'])
    
    return clean_dataset

##### Check if there are any nulls in any of the columns. sklearn freaks out when it sees nulls.